# Projet Informatique - Comics to Music

## GANs non-conditionnel

Dans le code, en voici les différents éléments : 
 - définition du générateur
 - définition du discriminateur
 - entraînement du modèle

In [1]:
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
import math
import matplotlib.pyplot as plt
import os

In [2]:
device = ''
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

### Générateur (sans conditionnement)

In [3]:
class NonConditionnalGenerator(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        
        '''On commence par définir les éléments structurels de notre générateur.
        Dans notre cas, on suppose d'abord qu'on veut générer vers un espace latent
        (génération conditionnée par la suite par un autre espace latent, plus des
        conditions supplémentaires).
        Le générateur va augmenter la dimension des données en opérant une déconvolution (Upsample, 2e article cGANs)
        Ici, on part d'un vecteur bruit z et on utilise une première couche linéaire pour 
        le mettre sous format-image en 2D. Dans cet exemple, on veut obtenir des images 28x28, donc on transforme z 
        en images 7x7, ce qui permettra facilement d'atteindre la dimension finale en réalisant deux Upsample de scale = 2
        '''

        self.z_dim = z_dim # dimension du vecteur bruit 
        self.init_dim = 7 # dans cet exemple, on veut partir d'une image 7x7
        self.num_ch = 128 # nombre de canaux arbitraire (dépend de l'espace latent, peut être des autres conditions)
        self.l1 = nn.Sequential( # première couche linéaire
            nn.Linear(z_dim, self.num_ch * self.init_dim * self.init_dim) # on change la dimension du bruit pour pouvoir le 
                                                                          #reshape en [num_ch] images de dimensions init_dim x init_dim
        )

        # Réseau convolutionnel pour générer une image 28x28  : G_non_conditionnel : bruit --> espace latent audio
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # passer de 7x7 à 14x14
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # on passe à 28x28
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        ######################
        
    def forward(self, x):
        # conversion du vecteur bruit
        output1 = self.l1(x) 
        # reshape en image
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 
        # CNN
        output = self.model(output2) 
        return output

In [4]:
z_dim = 100 # arbitraire
nc_generator = NonConditionnalGenerator(z_dim).to(device=device)

### Discriminateur (sans conditionnement)

In [5]:
class NonConditionnalDiscriminator(nn.Module):
    def __init__(self, num_ch):
        super().__init__()
        
        '''Le discriminateur va être entraîner pour différencier les données réelles (espace latent audio)
        des données générées (G(z), qui dans la version cGAN seront conditionnées par l'espace latent BD).
        En pratique, il ne prend ne prend en entrée que des vrais ou que des faux à la fois.
        Fonctionnement de l'entraînement : 
         - on génère les prédictions D(x_real) --> objectif D(x_real) = 1
         - on calcule la loss sur les prédictions réelles : loss_real = loss(D(x_real), 1)
         - on génère les prédictions D(x_fake) --> objectif D(x_fake) = 1
         - on calcule la loss sur les prédictions réelles : loss_fake = loss(D(x_fake), 1)
        Le discriminateur est donc entraîné sur loss_tot = (loss_real + loss_fake)
        
        Architecture : réseau convolutionnel classique, exemple dans le deuxième article sur les cGANs'''
        
        self.num_ch = num_ch # nombre de canaux, dépend de l'espace latent audio

        #  modèle de convolution classique, on cherche à densifier l'information des images à chaque couche
        self.model = nn.Sequential(
            nn.Conv2d(num_ch, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
            
            # A la fin, on réduit brusquement le nombre de canaux à 1, pour obtenir une prédiction scalire {0,1}.
            # Dans la version cGAN, c'est ici que l'on introduit la condition y !!!
            nn.Conv2d(512, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # prend un batch d'images d'entrée, sort un batch de 1 et de 0
        output = self.model(x).view(-1,1)
        return output

In [6]:
num_ch = 3 # arbitraire
nc_discriminator = NonConditionnalDiscriminator(num_ch).to(device=device)

### Entraînement (sans conditionnement)

In [8]:
# Dataset générés à partir de l'espace latent audio

batch_size = 32
train_loader = ???

SyntaxError: invalid syntax (844242149.py, line 4)

In [7]:
# Paramètres de l'entraînement

lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss() # ici, loss commune à D et G, à voir si on peut utiliser une loss différente

optimizer_nc_discriminator = torch.optim.Adam(nc_discriminator.parameters(), lr=lr)
optimizer_nc_generator = torch.optim.Adam(nc_generator.parameters(), lr=lr)

In [8]:
# Sauvegarde et récupération de G et D 
if os.path.isfile('discriminator2.pt') and os.path.isfile('generator2.pt'):
    nc_discriminator.load_state_dict(torch.load('./discriminator2.pt'))
    nc_generator.load_state_dict(torch.load('./generator2.pt'))   

else:
    for epoch in range(num_epochs):

        for n, (real_samples, _) in enumerate(train_loader):

            ### Entraînement nc_discriminator
            # On récupère les données réelles (reshape pour être sûr qu'elles soient acceptés par nc_D)
            real_samples = real_samples.view(-1, 1, 28, 28)

            # on génère les fake_samples à partir d'un bruit gaussien (pas de condition ici)
            z = torch.randn(batch_size, z_dim)
            fake_samples = torch.tanh(nc_generator(z))

            # Initialisation de la descente de gradient
            optimizer_nc_discriminator.zero_grad()

            # Loss sur les prédictions "données réelles"
            predict_real = nc_discriminator(real_samples)
            loss_real = loss_function(predict_real, torch.full((batch_size, 1), 1.0))

            # Loss sur les prédictions "données générées"
            predict_fake = nc_discriminator(fake_samples.detach())
            loss_fake = loss_function(predict_fake, torch.full((batch_size, 1), 0.0))

            # Loss complète
            loss_discriminator = (loss_real + loss_fake) / 2

            # Descente de gradient et mise à jour des poids w_D
            loss_discriminator.backward()
            optimizer_nc_discriminator.step()

            ### Entraînement nc_generator
            # Initialisation descente de gradient
            optimizer_nc_generator.zero_grad()

            # génération des fake_samples
            z = torch.randn(batch_size, 100)
            fake_samples_new = nc_generator(z)

            # récupération de la prédiction de D
            gen_prediction = nc_discriminator(fake_samples_new)

            # loss classique entre la prédiction D(fake_samples) et l'objectid vecteur de 1
            valid_labels = torch.full((batch_size, 1), 1.0)
            loss_generator = loss_function(gen_prediction, valid_labels)

            # OPTIONNEL --> diversity loss
            # on ajoute une diversity loss pour éviter de générer que des points identiques
            # sinon le générateur peut mapper à chaque fois vers une seule image qui à l'air vraie
            distances = torch.cdist(fake_samples_new, fake_samples_new, p=2) # renvoie la distance scalaire entre tous les éléments générés du batch
            mean_distance = torch.mean(distances)
            lambda_div = 0.2 # poids sur la loss
            loss_diversity = -lambda_div * mean_distance # on veut maximiser la distance donc on met un - devant

            # loss totale 
            loss_generator = loss_generator + loss_diversity
            
            loss_generator.backward()
            optimizer_nc_generator.step()

            # Show loss
            if n == batch_size - 2:
                print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
                print(f"Epoch: {epoch} Loss G.: {loss_generator}")

NameError: name 'train_loader' is not defined

## GANs condtionnels

Pas si différents des GANs non-conditionnels, il faut juste rajouter une condition $y$ à deux étapes du modèle.

In [9]:
class ConditionnalGenerator(nn.Module):
    def __init__(self, z_dim, y_dim):
        super().__init__()
        
        self.z_dim = z_dim # dimension du vecteur bruit 
        self.y_dim = y_dim # dimension du vecteur condition
        self.init_dim = 7 # dans cet exemple, on veut partir d"'une image 7x7
        self.num_ch = 128 # nombre de canaux arbitraire (dépend de l'espace latent, peut être des autres conditions)
        
        self.l1 = nn.Sequential( # première couche linéaire
            nn.Linear(self.z_dim + self.y_dim, self.num_ch * self.init_dim * self.init_dim) # cette fois, on part de 
                                                                                            # z_dim + y_dim
        )

        # Réseau convolutionnel identique au vanilla GAN
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # passer de 7x7 à 14x14
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # on passe à 28x28
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        ######################
        
    def forward(self, x):
        # Ici, x est la concaténation du bruit z et de la condition y
        # conversion du vecteur x
        output1 = self.l1(x) 
        # reshape en image
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 
        # CNN
        output = self.model(output2) 
        return output

In [10]:
z_dim = 100 # arbitraire
y_dim = 100 # arbitraire
c_generator = ConditionnalGenerator(z_dim, y_dim).to(device=device)

In [ ]:
class ConditionnalDiscriminator(nn.Module):
    def __init__(self, num_ch, latent_x_dim, y_cond_dim):
        super().__init__()
    
        self.num_ch = num_ch # nombre de canaux, dépend de l'espace latent audio
        self.y_cond_dim = y_cond_dim
        self.stride_size = 2 # valeur du stride pour connaître la dimension lors du reshape

        #  modèle de convolution classique, on cherche à densifier l'information des images à chaque couche
        self.model = nn.Sequential(
            nn.Conv2d(num_ch, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
        )

        # A cette étape, il faut applatir les données et y concaténer la condition y
        # calcul de la dimension du reshape :
        self.image_dim = int(latent_x_dim / (self.stride_size ** 3)) # on a appliqué 3 strides de 2
        self.flat_dim = 512 * self.image_dim * self.image_dim

        self.classifier = nn.Sequential(
            nn.Linear(self.flat_dim + self.y_cond_dim, 1), # ce n'est plus une convolution car vecteur plat
            nn.Sigmoid()
        )

    
    def forward(self, x, y):
        # on rentre x et y car on commence par traiter seulement x avant d'injecter y
        features = self.model(x)
        
        # Aplatissement vers la flat dimension
        features_flat = features.view(features.size(0), -1) 
        
        # Concaténation avec le vecteur condition y
        combined = torch.cat([features_flat, y], dim=1)
        
        # Prédiction finale
        output = self.classifier(combined)
        return output


In [44]:
num_ch = 3 # arbitraire
x_dim = 16 # arbitraire
y_cond_dim = 28 # arbitraire
c_discriminator = ConditionnalDiscriminator(num_ch, x_dim, y_cond_dim).to(device=device)

## GANs Cycle-Consistent

Dans notre cas, on veut que le GAN apprennent à générer des données dans l'espace latent $Y$ (audio) en étant influencé par l'espace latent $X$ (BDs). Pour cela, deux solutions : entraîner notre GAN avec des données réelles pairées, en conservant les paires dans les espaces latents $X$ et $Y$. Problème : auncun dataset existant possédant des paires d'audio et de BDs. On va donc chercher à entraîner notre GAN pour qu'il puisse associer des échantillons $x\in \{x_i\}$ à des échantillons $y\in\{y_i\}, de manière non supervisée, en détectant lui-même les caractéristiques pour associer une paire. On s'inspire de l'article *Unpaired Image-to-Image Translation
using Cycle-Consistent Adversarial Networks*, qui introduisent les *Cycle-Consistent GANs*. Ils utilisent ce réseau pour passer d'images à images. Dans notre cas, les espaces $X$ et $Y$ ne sont pas aussi similaires, il va peut être falloir injecter des conditions supplémentaires (structure, rythme, couleur, texte, sens --> CLIP), pour aider notre GAN à générer des paires qui ont du sens.

En pratique, on a besoin d'entraîner deux générateurs
$$
G : X \longrightarrow Y \\
F : Y \longrightarrow X
$$

Chacun associé à un discrimnateur, dont la loss est celle d'un GAN classique, avec une légère différence : on remplace la *negative log likelihood* par une erreur des moindres carrés. Cette loss s'est prouvée plus stable pour ce type de GAN :
$$
\begin{align}
\mathcal{L}_{LSGAN} (G, D_Y, X, Y) =\: &\mathbf{E}_{y\sim p_{data}(y)}[(D_Y(y) - 1)^2]\\ + \: &\mathbf{E}_{x\sim p_{data}(x)}[D_Y(G(x))^2]
\end{align}
$$ 

**Aparté : fondamentalement, je pense que $G(x)$, $D_Y(x)$, ou $F(y)$, etc... s'écrivent $G(z|x)$, $D_Y(z|x)$, ou $F(z|y)$, où $z$ est un bruit gaussien, et x, y sont les vecteurs conditions pour les GANs, qu'on concatène à z.**


Cette architecture permet d'ajouter une *cycle-loss*, qui détermine si les données sont bien reconstruites lors de l'application successive de $G$ et $F$ :
$$
\begin{align}
\mathcal{L}_{cyc} =\: &\mathbb{E}_{x\sim p_{data}(x)}[||F(G(x)) - x||]\\ + \:&\mathbb{E}_{y\sim p_{data}(y)}[||G(F(y)) - y||]
\end{align}
$$


Pour l'architecture, l'article s'inspire de Johnson et al. *Perceptual losses
for real-time style transfer and super-resolution* : 
1. Générateurs (structure encodeur-décodeur):
 - deux stride-2 convolutions
 - quelques blocs résiduels
 - deux stride-$\frac{1}{2}$ convolutions
 $\\$
 $\longrightarrow$ donc pour ces GANs là, on n'augmente pas la dimension comme vu dans les cGANs (on peut quand même modifier si jamais nos espaces $X$ et $Y$ n'ont pas les mêmes dimensions). Nos espaces latents auront peut être des dimensions différentes (par exemple : peut être besoin de plus grandes dimensions dans l'espace latent audio, si on utilise plein de conditions supplémentaires dans l'espace BD). 

 2. Discriminateurs :
 - 70x70 Patch-GANs : faits pour discriminer des grosses images (1080p par exemple). Ils ne discriminent que sur des portions de 70 par 70. Peut être pas nécessaire pour nous si on travaille sur des espaces latents de faibles dimensions. Mais peux justement permettre de travailler sur des espaces latents plus complexes sans trop augmenter la charge de calcul.

### Générateur $G$ : $X$(BDs) $\longrightarrow$ $Y$(audios)

$G$ part de $X$ l'espace latent BDs pour générer $Y$ l'espace latent audio. Comment on utilisera des conditions supplémentaires pour influencer la génération, on peut se permettre de ne garder que peut d'information dans $X$. Ainsi, on choisit de générer $X$ (partie VAE) comme un vecteur 1D plutôt que comme une image. On ne garde ainsi qu'une idée du style et on oublie les informations de structure spatiale qui seront utilisées plus efficacement dans les autres conditions.

In [45]:
class Generator_G(nn.Module):
    def __init__(self, z_dim, latent_x_dim, x_ch, init_dim):
        '''
        Inputs :
         - z_dim : dimension du vecteur bruit seed du générateur
        Dimensions des espaces latents: 
         - latent_x_dim : dimension de l'espace latent X d'où est tirée la condition du générateur G.
            On peut générer l'espace latent BDs comme un vecteur, car on ne veut l'utiliser que pour avoir une 
            notion du style
         - pas besoin de prendre latent_y_dim pour connaître la dimension de l'objectif si l'architecture
         est adaptée en fonction de la dimension de départ init_dim (voir juste en dessous)
        '''
        super().__init__()
        self.z_dim = z_dim # dimension du vecteur bruit 

        # dimensions des espaces X et Y (on suppose dim(X) < dim(Y))
        # G opère donc une déconvolution pour passer de X à Y
        self.x_dim = latent_x_dim # dimension du vecteur condition (espace BDs X)

        self.init_dim  = init_dim # dimension (H ou W) de l'image 2D à l'entrée du CNN
        # idéalement dim(Y) = dim_init * 2**(n_upsample) pour qu'on puisse facilement arriver à la bonne dimension (ici n_upsample = 2) 
        self.num_ch = x_ch # nombre de canaux arbitraire (dépend de l'espace latent qu'on choisit, peut être des autres conditions aussi)
        
        # Première couche linéaire
        self.l1 = nn.Sequential( 
            nn.Linear(self.z_dim + self.x_dim, self.num_ch * self.init_dim * self.init_dim) 
        )

        # Réseau convolutionnel --> déconvolution d'une petite dimension vers une grande dimension
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # (init_dim)x(init_dim) --> (2*init_dim)x(2*init_dim)
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # (2*init_dim)x(2*init_dim) --> (4*init_dim)x(4*init_dim)
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, cat_xz):
        # Ici, cat est la concaténation du bruit z et du vecteur x
        output1 = self.l1(cat_xz) 
        # reshape en image
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 
        # CNN
        output = self.model(output2) 
        # output correspond normalement à une approximation d'un échantillon de l'espace Y
        return output

In [46]:
z_dim = 100 # arbitraire
x_dim = 100 # dimension d'un vecteur
x_ch = 32
y_dim = 64 # arbitraire, tenseur 2D 64x64
init_dim = 64 // (2**2)
G = Generator_G(z_dim, x_dim, x_ch, init_dim).to(device=device)

### Discriminateur $D_Y$

$D_Y$ vérifie que $G(x) = \hat{y} \approx y$. Il prend donc également en condition le vecteur $x$, injecté à l'avant dernière couche du CNN. Le discriminateur va donc prendre en entrée des données 2D, on peut donc utiliser une structure patchGAN $N$x$N$, qui permettra de réduire la quantité de calculs de la discrimination, et d'avoir un espace latent $Y$ plus complexe.

Un discriminateur de type PatchGAN ne sort pas un scalaire de prédiction mais une grille de parédiction pour chaque patche de données. La couche linéaire à la fin du discriminateur conditionnel vu juste avant doit donc nécessairement disparaître car on veut une sortie 2D pour le discriminateur. Il faut donc trouver un autre moyen pour injecter la condition $X$. Une solution est de créer la taille des données $Y$ lors de passage successif dans les convolutions du CNN. A la couche où on a décidé d'injecter $X$, on utilise la taille de $Y$ pour étendre ou compresser $X$ (normalement étendre). on en fait une matrice de même taille que $Y$ en concaténant plusieurs vecteurs, et pusi on la concatène aux données entre deux convolutions.

In [47]:

class Discriminator_DY(nn.Module):
    def __init__(self, y_ch, latent_x_dim, ndf=64):
        """
        num_ch_y     : Canaux de l'image Y (ex: 3 pour RGB)
        latent_x_dim : Dimension du vecteur condition X (ex: 64, 128...)
        ndf          : Nombre de filtres de base (ex: 64)
        """
        super().__init__()

        # Partie avant la concaténation (sans la condition)
        self.block_y = nn.Sequential(
            nn.Conv2d(y_ch, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Dimension de la convolution après la concaténation
        input_dim_fused = ndf + latent_x_dim
        
        # CNN post-concaténation (avec la condition)
        self.block_fused = nn.Sequential(
            nn.Conv2d(input_dim_fused, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, kernel_size=4, stride=1, padding=1)
        )
        # pas de Flatten/Linear à la fin, ou de .view() dans le forward donc la sortie sera en 2D 
        # --> prédiction pour chaque patch d'une image Y

    def forward(self, y_input, x_condition):

        # on rentre Y dans la première partie du réseau
        feature_map = self.block_y(y_input)

        # on en ressort les dimensions
        batch_size, _, M_height, M_width = feature_map.size()
        
        # on reshape la condition X en fonction
        x_reshaped = x_condition.view(batch_size, -1, 1, 1)
        x_expanded = x_reshaped.expand(batch_size, -1, M_height, M_width)
        
        # On empile les features de l'image et la condition étendue
        combined_input = torch.cat([feature_map, x_expanded], dim=1)
        
        # partie finale (CNN)
        output = self.block_fused(combined_input)
        
        # l'output est donc  de dimension 
        return output

In [48]:
x_dim = 100 # dimension d'un vecteur
num_ch_y = 3 # arbitraire
DY = Discriminator_DY(num_ch_y, x_dim).to(device=device)

### Générateur $F$ : $Y$(audios) $\longrightarrow$ $X$(BDs)

Contrairement à $G$, qui développait les données, $F$ a pour but de compresser l'espace $Y$ vers $X$. On veut que les deux générateurs forment un équilibre lors de l'apprentissage. On a également les même dimensions que dans $G$, mais inversées. Il es dont logique que l'architecture de $F$ soit symétrique à celle de $G$.

In [49]:
class Generator_F(nn.Module):
    def __init__(self, z_dim, latent_x_dim, x_ch, y_ch, init_dim):
        '''
        Inputs :
         - latent_x_dim : dimension du vecteur de sortie (espace X)
         - latent_y_dim : nombre de canaux de l'espace latent d'entrée Y
         - z_dim        : dimension du bruit gaussien ajouté
         - init_dim     : dimension spatiale de référence
        '''
        super().__init__()
        self.x_dim = latent_x_dim
        self.y_ch = y_ch
        self.z_dim = z_dim     
        self.init_dim = init_dim
        self.internal_ch = x_ch

        # Le réseau prend maintenant en entrée canaux de Y + canaux de Z
        input_channels = self.y_ch + self.z_dim

        self.model = nn.Sequential(
            
            nn.Conv2d(input_channels, 256, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # equivalent à un Downsample
            nn.AvgPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(256, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, self.internal_ch, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(self.internal_ch),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.flat_dim = self.internal_ch * self.init_dim * self.init_dim
        
        self.final_head = nn.Sequential(
            nn.Linear(self.flat_dim, self.x_dim)
        )

    def forward(self, input_y, z):

        # comme on ajoute z à une matrice 2D, il faut l'étendre (comme la condition X dans DY)
        batch_size, _, h, w = input_y.size()
        z_expanded = z.view(batch_size, -1, 1, 1).expand(batch_size, -1, h, w)
        combined_input = torch.cat([input_y, z_expanded], dim=1)
        
        features = self.model(combined_input)
        features_flat = features.view(features.shape[0], -1)
        output_x = self.final_head(features_flat)
        
        return output_x

In [50]:
z_dim = 100 # arbitraire
x_dim = 100 # dimension d'un vecteur
x_ch = 32
y_ch = 64 # arbitraire, tenseur 2D 64x64
init_dim = 64 // (2**2)
F = Generator_F(z_dim, x_dim, x_ch, x_ch, init_dim).to(device=device)

### Discriminateur $D_X$

Ce discriminateur agit sur l'espace $X$. Les données sont donc 1D, et plus simples que pour $D_Y$, on n'a pas besoin d'utiliser un discriminateur de type PatchGAN. On utilise un discriminateur conditionnée classique.

In [ ]:
class Discriminator_DX(nn.Module):
    def __init__(self, x_ch, latent_x_dim, cond_dim, ndf=64):
        super().__init__()

        """
        num_ch_y     : Canaux de l'image Y (ex: 3 pour RGB)
        latent_x_dim : Dimension du vecteur condition X (ex: 64, 128...)
        ndf          : Nombre de filtres de base (ex: 64)
        """
        self.latent_x_dim = latent_x_dim
        self.cond_dim = cond_dim # taille de la condition
        self.x_ch = x_ch # nombre de canaux, dépend de l'espace latent audio

        self.stride_size = 2 # valeur du stride pour retrouver la dimension lors du reshape

        #  modèle de convolution classique, on cherche à densifier l'information des images à chaque couche
        self.model = nn.Sequential(
            nn.Conv2d(num_ch, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
        )

        # A cette étape, il faut aplatir les données et y concaténer la condition y
        # calcul de la dimension du reshape :
        self.image_dim = int(latent_x_dim / (self.stride_size ** 3)) # on a appliqué 3 strides de 2
        self.flat_dim = 512 * self.image_dim * self.image_dim

        self.classifier = nn.Sequential(
            nn.Linear(self.flat_dim + self.cond_dim, 1), # ce n'est plus une convolution car vecteur plat
            nn.Sigmoid()
        )

    
    def forward(self, x, y):
        # on rentre x et y car on commence par traiter seulement x avant d'injecter y
        features = self.model(x)
        
        # Aplatissement vers la flat dimension
        features_flat = features.view(features.size(0), -1) 
        
        # Concaténation avec le vecteur condition y
        combined = torch.cat([features_flat, y], dim=1)
        
        # Prédiction finale
        output = self.classifier(combined)
        return output
